In [ ]:
import pandas as pd
from datetime import datetime
from random import randint
from tqdm import tqdm
import numpy as np
from itertools import chain
from collections import Counter

## 1.2 Fingerprint Hashing

In [ ]:
transactions = pd.read_pickle('processed.pkl') #bank_transactions.csv dataset already processed, 'TransactionID' and 'CustomerID' columns are dropped

In [ ]:
transactions.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0
1,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0
2,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0
3,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0
4,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5


#### Hash function implemented for creating an hash value for each value in the dataframe

In [ ]:
# Hash function for datetime type
def hash_date(value):
    return int(pd.Timestamp(value).timestamp())

In [ ]:
#Hash function for string type
def hash_string(value,p):
    m =2^32 -1
    a =101
    b=456
    first  = sum([ ord(x)*(p^i) for i,x in enumerate(value)])%m
    return (a* first + b)%p

In [ ]:
#Hash function for float type
def hash_float(value):
    return int(value)

In [ ]:
#Collective Hash function, checks on the type of value parameter and applies the right hash function
def my_hash (value,p):
    if not isinstance(value,int):
            if isinstance(value,datetime): 
                value  = hash_date(value)
            elif isinstance(value,str): 
                value = hash_string(value,p)
            elif isinstance(value,float): 
                value = hash_float(value)
    return value

#### Initialization of the parameter for the MinHash algorithm

In [ ]:
p  = 125539 #prime number
N  = 100  #number of permutations
max_val = 2^32-1 #value used for the randomization in the creation of the permutations
permutations = [(randint(0,max_val), randint(0,max_val)) for _ in range(N)] #  N permutations of two integers a,b

#### MinHash function: creates a minHash signature for the transaction given as input.

The minHash function to create a minhash signature for each transaction has this parameters:
1. N hash functions that have this structure : f(value,a,b,p): return (a*value+b)%p, the N coefficients a and b are obtained by permutating two integers N times
2. p which is a prime number

The signature is represented as an array of length N, the i-th element is the minimum hash value obtained by applying the i-th hash function to every hash value of the transaction's features.

In [ ]:
def minHash(transaction,p,permutations):
    vec  = [float(np.inf) for _ in range(len(permutations))]

    for i,val in transaction.items():#Iterate over the features of the transaction 
        
        val =  my_hash(val,p)# Create Hash value for the features
        
        for perm_i,perm_vals in enumerate(permutations): #Apply N permutations to every hash value of the transaction
            
            a,b = perm_vals

            output = (a*val + b)%p 
       
            if(vec[perm_i] > output ): 
                vec[perm_i] = output

    return vec # Hash Signature for the given transaction

#### Create 'minhash' column for the bank_transactions.csv dataset

In [ ]:
transactions['minhash']  = [minHash(transactions.loc[i],p,permutations) for i in tqdm(range(len(new_data)))]

100%|██████████| 985322/985322 [12:09<00:00, 1350.76it/s]


In [ ]:
transactions.minhash

0         [720, 598, 214, 100, 357, 316, 248, 134, 229, ...
1         [5476, 15157, 5027, 2632, 2735, 7912, 936, 329...
2         [5476, 10580, 3686, 1836, 2735, 5524, 4154, 23...
3         [5476, 18431, 7702, 3844, 2735, 11548, 8672, 4...
4         [5476, 22126, 7702, 3844, 2735, 11548, 8097, 4...
                                ...                        
985317    [22392, 18400, 6406, 3196, 11193, 9604, 7214, ...
985318    [4742, 481, 3694, 1840, 2368, 5536, 4163, 2309...
985319    [21580, 17733, 6174, 3080, 10787, 9256, 6953, ...
985320    [24689, 23023, 3929, 4000, 14007, 12016, 9023,...
985321    [15616, 12834, 4470, 2228, 7805, 1839, 5036, 2...
Name: minhash, Length: 985322, dtype: object

## 1.3 Locality Sensitive Hashing

### LoadQuery dataset

In [ ]:
query =  pd.read_pickle('query.pkl')


In [ ]:
query.index = [ i for i in range(len(query))] #Adjust query dataset index

#### Create 'minhash' column for the query.csv dataframe

In [ ]:
query['minhash']  = [minHash(query.loc[i],p,permutations) for i in tqdm(range(len(query)))]

100%|██████████| 46/46 [00:00<00:00, 836.83it/s]


In [ ]:
query.head().minhash

0    [1840, 1518, 534, 260, 917, 796, 608, 334, 549...
1    [39435, 3087, 19822, 9904, 20987, 29728, 22307...
2    [15168, 12466, 4342, 2164, 7581, 6508, 4892, 2...
3    [18636, 23023, 8014, 4000, 9315, 10542, 9023, ...
4    [2260, 1863, 654, 320, 1127, 976, 743, 409, 66...
Name: minhash, dtype: object

### LSH 

INPUT
1. *dictc* a dictionary (empty or not), every key in the dict is a bin, the values are lists of the indexes of the rows that fell in the bin
2. *df* a dataframe that has a column 'minhash' containing the rows' signatures
3. *query* boolean parameter deafault if True it will insertthe index of the df row in the bucket in this format 'q<*index*>'. ù
4. *band* integer that represents the bandwidth of the algorithm, must be smaller than N. 

If you use a lower bandwidth each signature can fall in more bins so it will create more matching between the signatures.
If you use a higher bandwidth each signature can fall in less bins,so it will create much less matches between the signatures, but if two signatures fall in the same bin it will represent a stronger similarity.

OUTPUT
1. returns *dictc* after all the rows in the *df* dataframe are assigned to the correct bins

In [ ]:
# LSH algorithm implementation
def LSH(dictc,df,query,band):
    for i,hash_vec in enumerate(df.minhash):
        step = len(hash_vec)//band
        j = 0
        idx = 0
        while(j<len(hash_vec)):
            sub_vec = hash_vec[j:j+step]
            sep = '-'
            key  = sep.join([str(x) for x in sub_vec])+','+str(idx)
            if key not in dictc.keys():
                dictc[key]  = []
            if(query):
                dictc[key].append('q'+str(df.index))
            else:
                dictc[key].append(df.index)
            j += step
            idx +=1
    return dictc
       


#### fill_bin()
This function fills the bins of the query with the index every row passed in the df dataframe, if the row signatures falls in this bins.

In [ ]:
def fill_bins(query_bins,df,band):
    for i,hash_vec in enumerate(df.minhash):
        step = len(hash_vec)//band
        j = 0
        idx = 0
        while(j<len(hash_vec)):
            sub_vec = hash_vec[j:j+step]
            sep = '-'
            key  = sep.join([str(x) for x in sub_vec])+','+str(idx)
            if key in query_bins.keys():
                query_bins[key].append(i)
            j += step
            idx +=1
    return query_bins
       


#### Find the most similar Customer in the bank_transaction.csv file for the given Customer query

In [ ]:
# query is the entire row in the query.csv file with his minHash signature
# df is the dataframe where we want to find the most similar to the query, df comes with all the minHash signature for every Transaction in the dataset
# band: bandwidth parameter, sets the number of subdivisions in the minHash signature of the rows in df
#Changing the value of the bandwidth will create more matches, but the most similar will be unaffected
def check_similar(query,band,df):
    query_i= 'q'+str(query.index[0])
    bins = {} # Initialization of the dictionary that will represents the bins for the given query row
    LSH(bins,query,True,band) # LSH  algorithm: fills the dictionary creating the right bins for the given query row
    fill_bins(bins,df,band) #fill_bins:  function to insert in the right bins the index of every the df row 
    
    matches = [ x for v,x in bins.items()]
   
    matches  = [x for x in list(chain.from_iterable(matches)) if x != query_i and 'q' not in str(x)]
    c = Counter(matches)
    if( not c.most_common()):
        return 'NA'
    return c.most_common()[0][0] #Return the index in the df dataframe corresponding to the most similar Transaction

#### Find most similar for every row in query.csv in the bank_transaction.csv dataset

In [ ]:
matches =[]
for i in tqdm(query.index):
        matches.append(check_similar(query[i:i+1],4,transactions))

100%|██████████| 46/46 [31:46<00:00, 41.46s/it]


In [ ]:
query_results  = pd.DataFrame(columns=['query_index','match_df_index'])


In [ ]:
query_results['match_df_index'] = matches
query_results['query_index']  = query.index

In [ ]:
query_results

,query_index,match_df_index
0,0,701472
1,1,684212
2,2,296431
3,3,601262
4,4,8675
5,5,880492
6,6,68
7,7,13
8,8,563650
9,9,532514


In [ ]:
query

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),minhash
0,1978-07-27,M,DELHI,94695.61,2016-02-09,140310,65.00,"[1840, 1518, 534, 260, 917, 796, 608, 334, 549..."
1,1992-06-11,M,PANCHKULA,7584.09,2016-02-09,120214,6025.00,"[39435, 3087, 19822, 9904, 20987, 29728, 22307..."
2,1991-08-14,M,PATNA,7180.60,2016-10-08,221732,541.50,"[15168, 12466, 4342, 2164, 7581, 6508, 4892, 2..."
3,1987-03-01,M,CHENNAI,56847.75,2016-08-29,144138,1000.00,"[18636, 23023, 8014, 4000, 9315, 10542, 9023, ..."
4,1995-04-01,M,GURGAON,84950.13,2016-09-25,233309,80.00,"[2260, 1863, 654, 320, 1127, 976, 743, 409, 66..."
5,1981-10-01,M,WORLD TRADE CENTRE BANGALORE,23143.95,2016-11-09,192906,303.00,"[3211, 6992, 2438, 1212, 4249, 3652, 2750, 152..."
6,1976-09-20,F,CHITTOOR,15397.80,2016-08-28,92633,20.00,"[580, 483, 174, 80, 287, 256, 203, 109, 189, 4..."
7,1991-10-04,M,MOHALI,426.30,2016-02-08,203754,50.00,"[1420, 1173, 414, 200, 707, 616, 473, 259, 429..."
8,1990-03-19,M,MOHALI,4609.34,2016-08-26,184015,300.00,"[3533, 6923, 2414, 1200, 4207, 3616, 2723, 150..."
9,1970-12-19,M,SERAMPORE,6695988.46,2016-08-27,144030,299.00,"[8392, 2475, 2406, 1196, 4193, 3604, 2714, 150..."
